# Tests and Testability

**Writing and Testing Maintainable Code**

Here we will discuss and demonstrate testing a factorial function. `factorial(n)` is a function that computes the product of all values from 1 to n, so for instance:

| n | factorial(n) |
|--|--|
| 1 | 1 |
| 2 | 2 |
| 3 | 6 |
| 4 | 24 |

## Base Implementation

Let's assume our function looks like this:

In [1]:
def factorial(n):
    if n == 1:
        return n
    return n * factorial(n - 1)

## What to Test

You should always test the happy path, but also ways things can go wrong:

* happy path
  * base case
  * typical cases
* sad path
  * boundary conditions
  * invalid input
      * badly-formed input
      * malicious input
  * null values
  * exceptional cases


## Test Setup

To get started with unit testing, we simply need to import the `unittest` package. This is built-in to all modern versions of python.

In [2]:
import unittest

Tests can now be written by subclassing `unittest.TestCase`:

In [3]:
class TestFactorial(unittest.TestCase):
    
    def test_fail(self):
        self.fail('intentional failure')

## Happy Path Testing

Let's write the tests described above for base case and typical cases:

In [4]:
class TestFactorial(unittest.TestCase):
    
    def test_base_case(self):
        self.assertEqual(factorial(1), 1)
    
    def test_some_values(self):
        self.assertEqual(factorial(2), 2)
        self.assertEqual(factorial(3), 6)
        self.assertEqual(factorial(4), 24)
        self.assertEqual(factorial(10), 3628800)

**NOTE:** running unittests within Jupyter is different than running them on the command line.

Let's execute our test class so far:

In [5]:
def run_tests():
    test_suite = unittest.TestSuite()
    test_suite.addTest(unittest.makeSuite(TestFactorial))
    runner = unittest.TextTestRunner()
    runner.run(test_suite)

    
run_tests()

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


So far, so good!

## Sad Path Testing

```python
class TestFactorial(unittest.TestCase):

    # ...

    def test_boundary(self):
        self.assertEqual(factorial(0), 1)

    def test_negative_input(self):
        with self.assertRaises(ValueError):
            factorial(-1)

    def test_non_integer(self):
        with self.assertRaises(TypeError):
            factorial('bob')
```

Let's add those to the test class and re-run the tests:

In [6]:
class TestFactorial(unittest.TestCase):
    
    def test_base_case(self):
        self.assertEqual(factorial(1), 1)
    
    def test_some_values(self):
        self.assertEqual(factorial(2), 2)
        self.assertEqual(factorial(3), 6)
        self.assertEqual(factorial(4), 24)
        self.assertEqual(factorial(10), 3628800)

    def test_boundary(self):
        self.assertEqual(factorial(0), 1)

    def test_negative_input(self):
        with self.assertRaises(ValueError):
            factorial(-1)

    def test_non_integer(self):
        with self.assertRaises(TypeError):
            factorial('bob')

run_tests()

.EE..
ERROR: test_boundary (__main__.TestFactorial)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-72d9ae6f1c6e>", line 13, in test_boundary
    self.assertEqual(factorial(0), 1)
  File "<ipython-input-1-80b17e88eb28>", line 4, in factorial
    return n * factorial(n - 1)
  File "<ipython-input-1-80b17e88eb28>", line 4, in factorial
    return n * factorial(n - 1)
  File "<ipython-input-1-80b17e88eb28>", line 4, in factorial
    return n * factorial(n - 1)
  [Previous line repeated 2943 more times]
  File "<ipython-input-1-80b17e88eb28>", line 2, in factorial
    if n == 1:
RecursionError: maximum recursion depth exceeded in comparison

ERROR: test_negative_input (__main__.TestFactorial)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-72d9ae6f1c6e>", line 17, in test_negative_input
    factorial(-1)
  File "<ipython-input

Hmmm... we've got a couple errors. It looks like `factorial(0)` and `factorial(-1)` are not handled properly.

Here's the re-written factorial function to handle these cases:

In [7]:
def factorial(n):
    if n < 0:
        raise ValueError('n must be non-negative')
    if n == 0:
        return 1
    return n * factorial(n - 1)

run_tests()

.....
----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK


## Project Layout

Most projects are laid out with either the test directory as a subdirectory of the main project directory:

* `my_package/`
  * `setup.py`
  * `my_package/`
    * `my_module.py`
    * `tests/`
      * `my_module_test.py`
  * `README.md`
  * ...

... or the test directory as a sibling of the main project directory:


* `my_package/`
  * `setup.py`
  * `my_package/`
    * `my_module.py`
  * `tests/`
    * `my_module_test.py`
  * `README.md`
  * ...

There is not one correct way to do this, but the trend has become the second case above so that packages installed using [PyPI](http://pypi.org) don't include their internal tests; this helps to keep distributed package size small.

## Switching to `pytest`

That's all well and good, but `unittest` is a little clunky to set up and use. Many people prefer `pytest` because the testing code is a little more "Pythonic", though I've been using `unittest` for years without any troubles. However, by switching to `pytest` our tests would look something like this:

```python
import pytest
from factorial import factorial

class TestFactorial(object):

    def test_base_case(self):
        assert factorial(1) == 1

    def test_some_values(self):
        assert factorial(2) == 2
        assert factorial(3) == 6
        assert factorial(4) == 24
        assert factorial(10) == 3628800

    def test_boundary(self):
        assert factorial(0) == 1

    def test_negative_input(self):
        with pytest.raises(ValueError):
            factorial(-1)

    def test_non_integer(self):
        with pytest.raises(TypeError):
            factorial('bob')
```

... or we can convert them to plain functions, in which case they'll look like this:

```python
import pytest
from factorial import factorial

def test_base_case():
    assert factorial(1) == 1

def test_some_values():
    assert factorial(2) == 2
    assert factorial(3) == 6
    assert factorial(4) == 24
    assert factorial(10) == 3628800

def test_boundary():
    assert factorial(0) == 1

def test_negative_input():
    with pytest.raises(ValueError):
        factorial(-1)

def test_non_integer():
    with pytest.raises(TypeError):
        factorial('bob')
```